In [1]:
from io import StringIO

import os
import re
import PyPDF2
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure, LAParams
from ftlangdetect import detect
import pycld2 as cld2

In [2]:
def collect_clear_text(text_list):
    text_list = [re.sub(r'\s+', ' ', text_.strip()).strip() for text_ in text_list]
    text = ' '.join(text_list)
    return text.strip()
"""
def contents_references_rmv(text_per_page: dict,pages = 30, char_num = 100, years_num = 10):
    
    def less_char_drop(text,char_num):
        if len(text) < char_num: return True
    def dot_drop(text):
        if (len(text) > char_num) and ((text.count(".") > 40) or (text.count("…")> 10) or (text.count("_")> 10) or (text.count("-")>10)) : return True
    def referencesYear_drop(text):
        year_list = list(range(1900,2025))
        year_count = 0
        for year in year_list:
            if str(year) in text:
                year_count += 1
        if year_count > years_num: return True
    total_pages = len(text_per_page)
    obs_numbers = list(range(0,pages)) + list(range(total_pages-pages,total_pages))
    for page_num in obs_numbers:
        page_word = f'Page_{page_num}'
        if page_num < pages:
            try:
                if (less_char_drop(text_per_page[page_word],char_num)) or (dot_drop(text_per_page[page_word]))  : del text_per_page[page_word]   
            except KeyError:
                continue
        else:
            if (less_char_drop(text_per_page[page_word],char_num)) or (referencesYear_drop(text_per_page[page_word]))  : del text_per_page[page_word]
    return text_per_page
"""
def remove_eng(text_per_page: dict):
    dict_keys = list(text_per_page.keys())
    for keys in dict_keys: 
        text = text_per_page[keys]
        try:
            isReliable, textBytesFound, details = cld2.detect(text)
        except Exception as e:
            del text_per_page[keys]
            continue    
        lang = details[0][1]
        score = details[0][2]
        #print(keys, lang)
        if (lang == 'tr') and (score < 0.91):
            del text_per_page[keys]
        elif lang != "tr":
            del text_per_page[keys]
    return text_per_page

def remove_pageNum(text: str):
    text = text.strip(" ")
    text_list = text.split(" ")
    if text_list[0].strip().isnumeric():
        text_list.remove(text_list[0])
    if text_list[-1].strip().isnumeric():
        text_list.remove(text_list[-1])

    return " ".join(text_list).strip()


In [5]:
pdf_listDir = os.listdir("dergipark_download/DergiPark_PDF") 

for pdf in pdf_listDir:
    print(pdf)
    text_per_page = {}
    pdf_path = f"dergipark_download/DergiPark_PDF/{pdf}"
    try:
        for pagenum, page in enumerate(extract_pages(pdf_path, laparams = LAParams(line_margin = 2))):
            page_text = []
            page_elements = [(element.y1, element) for element in page._objs]
            page_elements.sort(key=lambda a: a[0], reverse=True)
            for i, component in enumerate(page_elements):
                element = component[1]
                if isinstance(element, LTTextContainer):
                    line_text = element.get_text()
                    page_text.append(line_text)
            dctkey = 'Page_'+str(pagenum)
            text_per_page[dctkey]= collect_clear_text(page_text)
    except Exception as e:
        print(e)
        continue
    rawTotal = len(text_per_page)
    #text_per_page = contents_references_rmv(text_per_page=text_per_page)
    text_per_page = remove_eng(text_per_page=text_per_page)

    clearedTotal = len(text_per_page)
    if clearedTotal < 2:
        print("Bozuk pdf:", pdf)
    print("PDF:", pdf, "Raw_Pages:", rawTotal, "Cleared_Total:", clearedTotal)
    with open(f"dergipark_download/DergiPark_TXT/{pdf[:-4]}.txt" , 'w+', encoding="utf-8") as txt:
        for text in text_per_page.values():
            txt.write(remove_pageNum(text))
            txt.write('\n')
    os.rename( f"dergipark_download/DergiPark_PDF/{pdf}", f"dergipark_download/DergiPark_PDF_fin/finished_{pdf}")

21. Y__zy__lda Ortado__u___da B__lgesel Rekabet_ T__rkiye ve M__s__r[#171216]-151135.pdf
Unexpected EOF - file corrupted
6328 Say__l__ Kamu Denet__ili__i Kurumu Kanunu ve Komisyon Raporlar__nda Yer Alan Ele__tiriler __er__evesinde Bir De__erlend (1).pdf
No /Root object! - Is this really a PDF?
6362 Say__l__ Sermaye Piyasas__ Kanunu'nda Sermaye Piyasas__ Arac__ Kavram__[#341603]-346953.pdf
PDF: 6362 Say__l__ Sermaye Piyasas__ Kanunu'nda Sermaye Piyasas__ Arac__ Kavram__[#341603]-346953.pdf Raw_Pages: 8 Cleared_Total: 8
6493 Say__l__ Kanunda __dari ve Cezai Sorumluluk[#374509]-398198.pdf
PDF: 6493 Say__l__ Kanunda __dari ve Cezai Sorumluluk[#374509]-398198.pdf Raw_Pages: 14 Cleared_Total: 12
6502 SAYILI T__KET__C__N__N KORUNMASI HAKKINDA KANUN A__ISINDAN T__KET__C__ KRED__S__ S__ZLE__MELER__NDE BANKAYA VER__LEN __AHS_.pdf
PDF: 6502 SAYILI T__KET__C__N__N KORUNMASI HAKKINDA KANUN A__ISINDAN T__KET__C__ KRED__S__ S__ZLE__MELER__NDE BANKAYA VER__LEN __AHS_.pdf Raw_Pages: 22 Cleared_Total: 2

The PDF <_io.BufferedReader name="dergipark_download/DergiPark_PDF/AB'NE __YEL__K S__REC__NDE T__RK__YE'DE KAYIT DI__I EKONOM__[#4345]-3752.pdf"> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


PDF: AB'de S______r ve Dana Eti Ortak Piyasa D__zeni ve T__rkiye A____s__ndan Bir De__erlendirme[#40588]-34141.pdf Raw_Pages: 7 Cleared_Total: 7
AB'NE __YEL__K S__REC__NDE T__RK__YE'DE KAYIT DI__I EKONOM__[#4345]-3752.pdf
PDF: AB'NE __YEL__K S__REC__NDE T__RK__YE'DE KAYIT DI__I EKONOM__[#4345]-3752.pdf Raw_Pages: 26 Cleared_Total: 25
AB'YE __YE OLAN __LKELERE YAPILAN DO__RUDAN YABANCI YATIRIMLARIN BORSADA ____LEM G__REN YERL__ ____RKETLER__N P__YASA DE__ER__ _.pdf
PDF: AB'YE __YE OLAN __LKELERE YAPILAN DO__RUDAN YABANCI YATIRIMLARIN BORSADA ____LEM G__REN YERL__ ____RKETLER__N P__YASA DE__ER__ _.pdf Raw_Pages: 9 Cleared_Total: 9
AB-28 __LKELER__NDE ULA__IM SEKT__R__ VE ULUSLARARASI T__CARET  LOJ__ST______N__N A__ ANAL__Z__ __LE __NCELENMES__[#699390]-994978.pdf
PDF: AB-28 __LKELER__NDE ULA__IM SEKT__R__ VE ULUSLARARASI T__CARET  LOJ__ST______N__N A__ ANAL__Z__ __LE __NCELENMES__[#699390]-994978.pdf Raw_Pages: 26 Cleared_Total: 22
AB-Akdeniz __lkeleri Aras__ndaki Tar__msal Ticaretin Ser

The PDF <_io.BufferedReader name='dergipark_download/DergiPark_PDF/Alternatif D__viz Kuru Sistemleri[#4246]-3654.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


PDF: Alt Dudak Kanserlerinde Rekonstr__ksiyon Se__enekleri[#913247]-1697987.pdf Raw_Pages: 8 Cleared_Total: 6
Alternatif D__viz Kuru Sistemleri[#4246]-3654.pdf
PDF: Alternatif D__viz Kuru Sistemleri[#4246]-3654.pdf Raw_Pages: 15 Cleared_Total: 15
Alternatif faizsiz sigortac__l__k uygulamalar__ ve gelir seviyesine g__re kamu bilincinin de__erlendirilmesi_ Ampirik analiz[#71.pdf
PDF: Alternatif faizsiz sigortac__l__k uygulamalar__ ve gelir seviyesine g__re kamu bilincinin de__erlendirilmesi_ Ampirik analiz[#71.pdf Raw_Pages: 22 Cleared_Total: 20
Alternatif Piyasa Oynakl__klar__nda Meydana Gelen K__r__lmalar__n ICSS Algoritmas__yla Belirlenmesi ve S__regenli__e Etkileri_ T.pdf
PDF: Alternatif Piyasa Oynakl__klar__nda Meydana Gelen K__r__lmalar__n ICSS Algoritmas__yla Belirlenmesi ve S__regenli__e Etkileri_ T.pdf Raw_Pages: 17 Cleared_Total: 16
Alternatif Sistematik Risk __l____tleri  ile Sermaye Varl__klar__ Fiyatlama  Modelinin Borsa __stanbul___da Test  Edilmesi[#6076.pdf
PDF: Alternati

...

PDF: T__RK__YE___DE VE D__NYADA SINIR __TES__ ELEKTRON__K T__CARET[#621928]-907494.pdf Raw_Pages: 12 Cleared_Total: 12
T__RK__YE___DE VE OECD __LKELER__NDE TASARRUF - YATIRIM A__IKLARI VE DI__ KAYNAK __HT__YACI[#4157]-3570.pdf
PDF: T__RK__YE___DE VE OECD __LKELER__NDE TASARRUF - YATIRIM A__IKLARI VE DI__ KAYNAK __HT__YACI[#4157]-3570.pdf Raw_Pages: 32 Cleared_Total: 29
T__RK__YE___DE VERG__ AFLARI VE 6111 SAYILI KANUNUN DE__ERLEND__R__LMES___ BAYBURT __L__ __RNE____[#66839]-56698.pdf
PDF: T__RK__YE___DE VERG__ AFLARI VE 6111 SAYILI KANUNUN DE__ERLEND__R__LMES___ BAYBURT __L__ __RNE____[#66839]-56698.pdf Raw_Pages: 10 Cleared_Total: 10
T__RK__YE___DE VERG__ AFLARININ VERG__ ADALET__ VE M__KELLEFLER __ZER__NE ETK__LER__[#8659]-7888.pdf
PDF: T__RK__YE___DE VERG__ AFLARININ VERG__ ADALET__ VE M__KELLEFLER __ZER__NE ETK__LER__[#8659]-7888.pdf Raw_Pages: 22 Cleared_Total: 21
T__RK__YE___DE VERG__ DENET__M__ VE GEL__RLER__ __ZER__NE B__R __NCELEME[#242479]-210884.pdf
PDF: T__RK__YE___DE VERG_

Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


PDF: VERG__ KA__AK__ILI__I VE KAYIT DI__I EKONOM__[#547190]-703422.pdf Raw_Pages: 7 Cleared_Total: 7
VERG__ KA__IRMA ET______ __ZER__NE T__RK VE RUS ____LETME Y__NET__M__ ____RENC__LER__N__N KAR__ILA__TIRMALI B__R ANAL__Z__[#222889]-194438.pdf
Bozuk pdf: VERG__ KA__IRMA ET______ __ZER__NE T__RK VE RUS ____LETME Y__NET__M__ ____RENC__LER__N__N KAR__ILA__TIRMALI B__R ANAL__Z__[#222889]-194438.pdf
PDF: VERG__ KA__IRMA ET______ __ZER__NE T__RK VE RUS ____LETME Y__NET__M__ ____RENC__LER__N__N KAR__ILA__TIRMALI B__R ANAL__Z__[#222889]-194438.pdf Raw_Pages: 20 Cleared_Total: 1
VERG__ K__LT__R__ VE G__STERGELER___ T__RK__YE __ZER__NE B__R DE__ERLEND__RME[#353846]-364907.pdf
PDF: VERG__ K__LT__R__ VE G__STERGELER___ T__RK__YE __ZER__NE B__R DE__ERLEND__RME[#353846]-364907.pdf Raw_Pages: 14 Cleared_Total: 14
VERG__ MEVZUATININ KARMA__IKLI__I VE UZLA__MA Y__NTEM__ BA__LAMINDA VERG__ UYUMUNUN DE__ERLEND__R__LMES__[#333195]-332674.pdf
PDF: VERG__ MEVZUATININ KARMA__IKLI__I VE UZLA__MA Y__NTEM__ BA_

The PDF <_io.BufferedReader name='dergipark_download/DergiPark_PDF/YOLSUZLUKLARIN B__T__E A__IKLARI __ZER__NDEK__ ETK__S__[#4193]-3603.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


PDF: YOLSUZLUKLA M__CADELE  BOYUTUYLA  ET__K ve ET__K K__LT__R__[#240454]-208903.pdf Raw_Pages: 10 Cleared_Total: 10
YOLSUZLUKLARIN B__T__E A__IKLARI __ZER__NDEK__ ETK__S__[#4193]-3603.pdf
PDF: YOLSUZLUKLARIN B__T__E A__IKLARI __ZER__NDEK__ ETK__S__[#4193]-3603.pdf Raw_Pages: 12 Cleared_Total: 9
YOLSUZLUKLARIN VERG__ GEL__RLER__ __ZER__NDEK__  ETK__S___ D__NAM__K PANEL VER__ ANAL__Z__[#1043025]-2152006.PDF
PDF: YOLSUZLUKLARIN VERG__ GEL__RLER__ __ZER__NDEK__  ETK__S___ D__NAM__K PANEL VER__ ANAL__Z__[#1043025]-2152006.PDF Raw_Pages: 11 Cleared_Total: 10
Yolsuzlu__un Nedenleri, Etkileri ve Yolsuzlukla M__cadelede Alternatif Bir Yakla____m Olarak Y__neti__im[#8064]-7294.pdf
Bozuk pdf: Yolsuzlu__un Nedenleri, Etkileri ve Yolsuzlukla M__cadelede Alternatif Bir Yakla____m Olarak Y__neti__im[#8064]-7294.pdf
PDF: Yolsuzlu__un Nedenleri, Etkileri ve Yolsuzlukla M__cadelede Alternatif Bir Yakla____m Olarak Y__neti__im[#8064]-7294.pdf Raw_Pages: 24 Cleared_Total: 0
Yolsuzlu__un Vergi Gelirleri _